Neslihan Gül 2021107015

**QUESTION 1**

Question 1-a)

In [1]:
def get_page(url):
    try:
        import urllib.request
        page = urllib.request.urlopen(url).read()
        page = page.decode("utf-8")
        return page
    except:
        return ""

def get_next_target(page):
    start_link = page.find('<a href=')
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1:end_quote]
    return url, end_quote

def get_all_links(page):
    links = []
    while True:
        url, endpos = get_next_target(page)
        if url:
            links.append(url)
            page = page[endpos:]
        else:
            break
    return links

def union(p, q):
    for e in q:
        if e not in p:
            p.append(e)

def add_toIndex(index, keyword, url):
    if keyword in index:
        index[keyword].append(url)
    else:
        index[keyword] = [url]

def getclearpage(content):
    title = content[content.find("<title>") + 7:content.find("</title>")]
    body = content[content.find("<body>") + 6:content.find("</body>")]
    while body.find(">") != -1:
        start = body.find("<")
        end = body.find(">")
        body = body[:start] + body[end + 1:]
    return title + body

def addPageToIndex(index, url, content):
    content = getclearpage(content)
    words = content.split()
    for word in words:
        add_toIndex(index, word, url)

def crawlWeb(seed):
    tocrawl = [seed]
    crawled = []
    index = {}
    graph = {}
    while tocrawl:
        page = tocrawl.pop()
        if page not in crawled:
            content = get_page(page)
            addPageToIndex(index, page, content)
            outlinks = get_all_links(content)
            graph[page] = outlinks
            union(tocrawl, outlinks)
            crawled.append(page)
    return index, graph

def lookup(index, keyword):
    if keyword in index:
        return index[keyword]
    else:
        return None

index1, graph1 = crawlWeb("https://searchengineplaces.com.tr/")


# Explanation:
# The graph represents the structure of the web as a directed graph.
# Each node in the graph is a URL (web page), and corresponding value is a list of other URLs (outlinks) that the page links to.
# This structure helps us understand how pages are interconnected.

Question 1-b)

In [15]:
print(f"The graph has {len(graph1)} elements. These are:")
for i, (page, outlinks) in enumerate(graph1.items(), 1):
    print(f"{i}.\t[{page}] : {outlinks}")

The graph has 10 elements. These are:
1.	[https://searchengineplaces.com.tr/] : ['http://www.searchengineplaces.com.tr/travel_guide.html']
2.	[http://www.searchengineplaces.com.tr/travel_guide.html] : ['http://www.searchengineplaces.com.tr/ankara.html', 'http://www.searchengineplaces.com.tr/konya.html', 'http://www.searchengineplaces.com.tr/istanbul.html', 'http://www.searchengineplaces.com.tr/oktayrecommends.html', 'http://www.searchengineplaces.com.tr/seymarecommends.html']
3.	[http://www.searchengineplaces.com.tr/seymarecommends.html] : ['http://www.searchengineplaces.com.tr/oktayrecommends.html', 'http://www.searchengineplaces.com.tr/konya.html']
4.	[http://www.searchengineplaces.com.tr/oktayrecommends.html] : ['http://www.searchengineplaces.com.tr/istanbul.html']
5.	[http://www.searchengineplaces.com.tr/istanbul.html] : ['http://www.searchengineplaces.com.tr/maidens_tower.html', 'http://www.searchengineplaces.com.tr/galata_tower.html']
6.	[http://www.searchengineplaces.com.tr/gala

QUESTION 1-c)

In [3]:
def computeRanks(graph):
    d = 0.8  # Damping factor
    N = len(graph)  # Number of pages
    numloops = 10  # Number of iterations to refine ranks
    ranks = {}

    # Initialize ranks: each page gets an equal share
    for page in graph:
        ranks[page] = 1 / N

    for i in range(numloops):
        newranks = {}
        for page in graph:
            newrank = (1 - d) / N  # Base rank contribution
            for node in graph:
                if page in graph[node]:
                    newrank += d * (ranks[node] / len(graph[node]))
            newranks[page] = newrank
        ranks = newranks  # Update ranks after each iteration

    return ranks

ranks1 = computeRanks(graph1)

for page, rank in ranks1.items():
    print(f"The rank of the page {page} : {rank}")

The rank of the page https://searchengineplaces.com.tr/ : 0.019999999999999997
The rank of the page http://www.searchengineplaces.com.tr/travel_guide.html : 0.14780429869056003
The rank of the page http://www.searchengineplaces.com.tr/seymarecommends.html : 0.073060618584064
The rank of the page http://www.searchengineplaces.com.tr/oktayrecommends.html : 0.073060618584064
The rank of the page http://www.searchengineplaces.com.tr/istanbul.html : 0.17460832634470402
The rank of the page http://www.searchengineplaces.com.tr/galata_tower.html : 0.09025810358272002
The rank of the page http://www.searchengineplaces.com.tr/maidens_tower.html : 0.09025810358272002
The rank of the page http://www.searchengineplaces.com.tr/konya.html : 0.073060618584064
The rank of the page http://www.searchengineplaces.com.tr/mevlana.html : 0.04928445079552
The rank of the page http://www.searchengineplaces.com.tr/ankara.html : 0.043776167788544


QUESTION 1-d)

In [4]:
def rankedLookup(index, keyword, graph):
    pages = index.get(keyword, [])
    ranks = computeRanks(graph)
    unique_pages = set(pages)
    return sorted(unique_pages, key=lambda page: ranks.get(page, 0), reverse=True)

results = rankedLookup(index1, "in", graph1)
for result in results:
    print(result)

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html


QUESTION 1-e)

In [5]:
def lookup(index, keyword, *args):
    if len(args) == 0:

        pages = set(index.get(keyword, []))
        return list(pages)
    elif len(args) == 2:

        graph, computeProcedure = args
        ranks = computeProcedure(graph)
        pages = set(index.get(keyword, []))
        return sorted(pages, key=lambda page: ranks.get(page, 0), reverse=True)
    elif len(args) == 1:

        return "Warning: Unexpected number of arguments. Provide either 2 or 4 arguments."
    else:

        raise ValueError("Invalid number of arguments provided to the lookup function.")


In [6]:
see = lookup(index1, "in", graph1, computeRanks)
for e in see:
    print(e)

http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/mevlana.html


In [38]:
see1 = lookup(index1, "in")
for e in see1:
    print(e)

http://www.searchengineplaces.com.tr/konya.html
http://www.searchengineplaces.com.tr/galata_tower.html
http://www.searchengineplaces.com.tr/istanbul.html
http://www.searchengineplaces.com.tr/travel_guide.html
http://www.searchengineplaces.com.tr/maidens_tower.html
http://www.searchengineplaces.com.tr/mevlana.html


In [8]:
assert rankedLookup(index1, "in", graph1) == lookup(index1, "in", graph1, computeRanks)

**QUESTION 2**

In [17]:
import requests
import random

In [34]:
def download_file_from_drive(file_url, colab_path):
    if "drive.google.com" in file_url:
        file_id = file_url.split('/d/')[1].split('/')[0]
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
    else:
        raise ValueError("The provided URL is not a valid Google Drive link.")

    response = requests.get(download_url, stream=True)

    if response.status_code == 200:
        with open(colab_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"File successfully downloaded to {colab_path}")
    else:
        raise Exception("Failed to download the file. Check the URL or file permissions.")

colab_path = "/content/words.txt"
file_url = "https://drive.google.com/file/d/1cjCZ-5uAHVFLXLV6HuSM98_QblWaXrBZ/view?usp=share_link"
download_file_from_drive(file_url,colab_path)


WORDLIST_FILENAME = "words.txt"

def load_words():

  inFile = open(WORDLIST_FILENAME, 'r')
  line = inFile.readline()
  word_list = line.split()
  print("  ", len(word_list), "words loaded.")
  return word_list

word_list = load_words()

File successfully downloaded to /content/words.txt
   55900 words loaded.


In [30]:
#Helper functions
def word_select(length):
    valid_words = [word for word in word_list if len(word) == length]
    return random.choice(valid_words)

def updatelist1(guess, feedback):
    return [word for word in word_list if all(
        (feedback[i] == 'g' and word[i] == guess[i]) or
        (feedback[i] == 'y' and guess[i] in word and word[i] != guess[i]) or
        (feedback[i] == 'b' and guess[i] not in word)
        for i in range(len(guess))
    )]

def updatelist2(guess, feedback):
    return [word for word in word_list if sum(
        (feedback[i] == 'g' and word[i] == guess[i]) +
        (feedback[i] == 'y' and guess[i] in word and word[i] != guess[i]) +
        (feedback[i] == 'b' and guess[i] not in word)
    ) == len(guess)]

def updatelist3(guess, feedback):
    remaining_words = []
    for word in word_list:
        matches = True
        for i in range(len(guess)):
            if feedback[i] == 'g' and word[i] != guess[i]:
                matches = False
            elif feedback[i] == 'y' and (guess[i] not in word or word[i] == guess[i]):
                matches = False
            elif feedback[i] == 'b' and guess[i] in word:
                matches = False
        if matches:
            remaining_words.append(word)
    return remaining_words

def cleanwordlist(length):
    global word_list
    word_list = [word for word in word_list if len(word) == length]

In [31]:
def wordle(secret_word, guess):
    feedback = []
    list1 = [None] * len(secret_word)  # Letters in the correct position
    list2 = [None] * len(secret_word)  # Letters in the wrong position.
    list3 = []                         # Letters not present in the word.

    # Feedback for the guess
    for i in range(len(guess)):
        if guess[i] == secret_word[i]:
            feedback.append('g')
            list1[i] = guess[i]  # Update correct letter in the correct position
        elif guess[i] in secret_word:
            feedback.append('y')
            list2[i] = guess[i]  # Update correct letter in the wrong position
        else:
            feedback.append('b')
            if guess[i] not in list3:
                list3.append(guess[i])  # Add to letters not in the word

    return feedback, list1, list2, list3

In [32]:
def generate_guess(length, list1, list2, list3):
    possible_words = word_list

    # Filter based on list1
    for i, char in enumerate(list1):
        if char is not None:  # If we know the correct character at this position
            possible_words = [word for word in possible_words if word[i] == char]

    # Filter based on list2
    for i, char in enumerate(list2):
        if char is not None:  # If we know the character is in the word but not in this position
            possible_words = [word for word in possible_words if char in word and word[i] != char]

    # Filter based on list3
    for char in list3:
        possible_words = [word for word in possible_words if char not in word]

    # Return a random word from the refined list
    return random.choice(possible_words) if possible_words else None

In [37]:
#OPTIMIZE WORDLE PROCEDURE
def optimize_wordle(secret_word):

    cleanwordlist(len(secret_word))
    attempts = 0
    guesses = []
    list1 = [None] * len(secret_word)
    list2 = [None] * len(secret_word)
    list3 = []

    while True:
        attempts += 1
        guess = generate_guess(len(secret_word), list1, list2, list3)

        if not guess:
            print("No valid words left to guess! Unable to complete the game.")
            break

        guesses.append(guess)
        print(f"Computer's guess is: {guess}\n")

        feedback, list1, list2, list3 = wordle(secret_word, guess)

        if ''.join(feedback) == 'g' * len(secret_word):  # If all positions are correct
            print(f"The computer guessed the word in {attempts} attempts.")
            print(f"List of the computer's guesses: {guesses}")
            break

# Example of calling the function
secret_word = "gong"
optimize_wordle(secret_word)

Computer's guess is: quiz

Computer's guess is: bash

Computer's guess is: vote

Computer's guess is: dodo

Computer's guess is: coop

Computer's guess is: holy

Computer's guess is: mown

Computer's guess is: pone

Computer's guess is: song

Computer's guess is: bong

Computer's guess is: gong

The computer guessed the word in 11 attempts.
List of the computer's guesses: ['quiz', 'bash', 'vote', 'dodo', 'coop', 'holy', 'mown', 'pone', 'song', 'bong', 'gong']
